**SOURCES**

https://spark.apache.org/docs/latest/api/python/index.html

https://www.kaggle.com/code/nezarabdilahprakasa/sentiment-analysis-using-pyspark-big-data


In [10]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.classification import NaiveBayes, NaiveBayesModel

In [2]:
spark = SparkSession.builder.appName("main_test").getOrCreate()

24/06/13 13:07:57 WARN Utils: Your hostname, halloa93 resolves to a loopback address: 127.0.1.1; using 192.168.55.114 instead (on interface wlp2s0)
24/06/13 13:07:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/13 13:07:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


**NEW TWEETS DOWNLOAD AND MODEL TESTING**

In [3]:
from ntscraper import Nitter
from pprint import pprint

In [4]:
scraper = Nitter(log_level = 1, skip_instance_check = False)

Testing instances:  92%|██████████████████████▏ | 71/77 [00:49<00:04,  1.35it/s]

13-Jun-24 13:08:52 - Certificate did not match expected hostname: nt.ggtyler.dev. Certificate: {'subject': ((('commonName', '4g.ggtyler.dev'),),), 'issuer': ((('countryName', 'US'),), (('organizationName', "Let's Encrypt"),), (('commonName', 'R3'),)), 'version': 3, 'serialNumber': '03A4459CE6DA31CFF555DB78FF3F35BFEA88', 'notBefore': 'May 14 10:55:44 2024 GMT', 'notAfter': 'Aug 12 10:55:43 2024 GMT', 'subjectAltName': (('DNS', '4g.ggtyler.dev'),), 'OCSP': ('http://r3.o.lencr.org',), 'caIssuers': ('http://r3.i.lencr.org/',)}


Testing instances:  95%|██████████████████████▊ | 73/77 [00:50<00:03,  1.27it/s]

13-Jun-24 13:08:54 - Certificate did not match expected hostname: nitter.uni-sonia.com. Certificate: {'subject': ((('commonName', '*.xserver.jp'),),), 'issuer': ((('countryName', 'JP'),), (('organizationName', 'CloudSecure Corporation'),), (('commonName', 'CloudSecure RSA Domain Validation Secure Server CA 2'),)), 'version': 3, 'serialNumber': 'ACA67AD2030638EE2DCE8E845B8299A6', 'notBefore': 'Mar 11 00:00:00 2024 GMT', 'notAfter': 'Apr 11 23:59:59 2025 GMT', 'subjectAltName': (('DNS', '*.xserver.jp'), ('DNS', 'xserver.jp')), 'OCSP': ('http://ocsp.sectigo.com',), 'caIssuers': ('http://crt.sectigo.com/CloudSecureRSADomainValidationSecureServerCA2.crt',)}


Testing instances:  99%|███████████████████████▋| 76/77 [00:55<00:01,  1.31s/it]

13-Jun-24 13:08:58 - Certificate did not match expected hostname: nitter.tinfoil-hat.net. Certificate: {'subject': ((('commonName', 'jelly.tinfoil-hat.de'),),), 'issuer': ((('countryName', 'US'),), (('organizationName', "Let's Encrypt"),), (('commonName', 'R3'),)), 'version': 3, 'serialNumber': '044FDE3E7089FB997C3D8AFDE2412CE51554', 'notBefore': 'May 15 09:29:23 2024 GMT', 'notAfter': 'Aug 13 09:29:22 2024 GMT', 'subjectAltName': (('DNS', 'jelly.tinfoil-hat.de'),), 'OCSP': ('http://r3.o.lencr.org',), 'caIssuers': ('http://r3.i.lencr.org/',)}


Testing instances: 100%|████████████████████████| 77/77 [00:55<00:00,  1.39it/s]


In [5]:
tweets = scraper.get_tweets("elonmusk", mode = "user", number = 10)

13-Jun-24 13:09:05 - No instance specified, using random instance https://nitter.privacydev.net
13-Jun-24 13:09:09 - Current stats for elonmusk: 10 tweets, 0 threads...


In [6]:
pprint(tweets)

{'threads': [],
 'tweets': [{'date': 'Jun 11, 2024 · 7:10 PM UTC',
             'external-link': '',
             'gifs': [],
             'is-pinned': True,
             'is-retweet': False,
             'link': 'https://twitter.com/elonmusk/status/1800606627319456065#m',
             'pictures': [],
             'quoted-post': {'date': 'Jun 11, 2024 · 7:07 PM UTC',
                             'gifs': [],
                             'link': 'https://twitter.com/cb_doge/status/1800605859669795263#m',
                             'pictures': [],
                             'text': 'To share 𝕏.com links with others, use '
                                     "the ''Share icon'' located at the bottom "
                                     'right of the post.',
                             'user': {'avatar': 'https://pbs.twimg.com/profile_images/1498070100393754625/C2V-fbll_mini.jpg',
                                      'name': 'DogeDesigner',
                                      'pr

In [7]:
sentence = tweets['tweets'][0]['quoted-post']['text']

In [8]:
print(sentence)

To share 𝕏.com links with others, use the ''Share icon'' located at the bottom right of the post.


In [11]:
from pyspark.ml import PipelineModel
from pyspark.ml.feature import StopWordsRemover, HashingTF, Tokenizer
from pyspark.sql import Row

#manual test
#sentence = "happy adventure music"

#transforming sentecne into row and dataframe
row = Row(id=0, text=sentence)

df = spark.createDataFrame([row])

loaded_model_lr = LogisticRegressionModel.load("lr_model_reference")
loaded_model_nb = NaiveBayesModel.load("nb_model_reference")

#tokenization of new data
tokenizer = Tokenizer(inputCol="text", outputCol="words")
tokenized_sentence = tokenizer.transform(df)

#stop words removing
swr = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filtered_sentence = swr.transform(tokenized_sentence)

#words hashing
hashTF = HashingTF(inputCol="filtered_words", outputCol="features")
hashed_sentence = hashTF.transform(filtered_sentence)

#tweet check
hashed_sentence.select("text", "features").show(truncate=False)


+--------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                              |features                                                                                                                      |
+--------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------+
|To share 𝕏.com links with others, use the ''Share icon'' located at the bottom right of the post.|(262144,[4309,45152,64859,80496,91064,98717,103377,110369,206556,227107,229166],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
+----------------------------------------------------------------------------------------

In [12]:
#tweet evaluation with lr model
predictions_lr = loaded_model_lr.transform(hashed_sentence)

In [13]:
#tweet evaluation with nb model
predictions_nb = loaded_model_nb.transform(hashed_sentence)

In [14]:
#lr model evaluation output of downloaded tweet
predicted_sentiment_lr = predictions_lr.select("prediction").collect()[0][0]
if predicted_sentiment_lr == 1.0:
    print("sentiment is positive")
else:
    print("sentiment is negative")

24/06/13 13:11:30 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


sentiment is negative


In [15]:
#nb model evaluation output of downloaded tweet
predicted_sentiment_nb = predictions_nb.select("prediction").collect()[0][0]
if predicted_sentiment_nb == 1.0:
    print("sentiment is positive")
else:
    print("sentiment is negative")

24/06/13 13:11:33 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


sentiment is negative
